# Inspecting the results of `bmfm-multi-omics` for visualiing the embeddings using t-SNE and PCA obtained from BMFM-DNA

In this tutorial we look at inspecting the results of the zero-shot prediction created in tutorial 1. We do this by loading the results of the data and then using the helper functions packaged in the `evaluation` module to help extract and interpret the results of the model.

In [1]:

import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc

from bmfm_targets.evaluation.embeddings import generate_clusters

/proj/bmfm/users/sanjoy/miniforge3/envs/bmfm-targets/lib/python3.11/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [ ]:
# Ran the embedding generation using the following command on outer sourced version:
# export WORKING_DIR=...
# export INPUT_DIR=...
# bsub -M 50G -n 8 -gpu "num=1:mode=exclusive_process" -o mpra_k562.out -e mpra_k562.err -W 24:00 bash -c 'bmfm-targets-run -cn dna_predict working_dir=$WORKING_DIR checkpoint=ibm-research/biomed.dna.ref.modernbert.113m.v1 accelerator=cpu input_directory=$INPUT_DIR label_column_name=mean_value dataset_name=mpra_k562_original input_filename=test.csv'

# Define a function to get the plots for each settings.

In [11]:
def plot_umap(results, labels, reference_labels, output_filename=None):
    # Create an AnnData object
    results.index.name = "dnaseq_id"
    results.index = results.index.astype(str)
    results.columns = [ f"dim_{i}" for i in range(results.shape[1])]

    adata = sc.AnnData(X=results)
    adata.X = adata.X.astype("float64")

    print("adata:", adata)
    reference_labels.index.name = "dnaseq_id"
    adata.obs = reference_labels  
    adata.obs.head()
 
    bmfm_adata = adata
    bmfm_adata.obs.head()

    print(adata.obs.head(), adata.var.head(), adata.X.shape)
    
    # Generate clusters
    bmfm_adata = generate_clusters(
        bmfm_adata,
        n_components=5,
        label=labels,
        clustering_method="kmeans",
        n_clusters=11,
    )

    #labels = "label"
    import seaborn as sns

    custom_params = {"axes.spines.right": False, "axes.spines.top": False}
    sns.set_theme(style="ticks", rc=custom_params)

    title = (f"UMAP - {labels}",)
    
    sc.tl.umap(bmfm_adata, n_components=2)
    sc.pl.embedding(
        bmfm_adata,
        basis="umap",
        title=title,
        color=labels,
        color_map="magma",
        #palette=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"],  # your custom colors
        show=False,
    )

    plt.show()


    # # Try t_SNE now...
    from sklearn.manifold import TSNE

    tsne = TSNE(n_components=2, random_state=42, perplexity=30, learning_rate=200)
    embeddings_2d = tsne.fit_transform(bmfm_adata.X)

    # 2. Plot
    plt.figure(figsize=(8, 6))
    colors = bmfm_adata.obs[labels]
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], s=10, alpha=0.7, c=colors, cmap="magma")
    plt.title(f"t-SNE Visualization of {output_filename} Embeddings")
    plt.xlabel("t-SNE 1")
    plt.ylabel("t-SNE 2")
    plt.colorbar(label=labels)
    plt.show()

    
    import scipy.stats as stats
    print(stats.pearsonr(embeddings_2d[:, 0],bmfm_adata.obs[labels])[0])
    print(stats.pearsonr(embeddings_2d[:, 1],bmfm_adata.obs[labels])[0])

# CAGI dataset

In [ ]:
input_directory="/proj/bmfm/datasets/omics/genome/finetune_datasets/snv_mpra_cagi_regression/"
reference_labels = pd.read_csv(input_directory + "merged_data_imputed_06-16-2025_flattened.csv")


results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/cagi_whole_hic/ref_ckpt/embeddings.csv", index_col=0, header=None)
print(results.shape)

# Take only the mut_seqs which is the last 1/3 rows of the reference labels file.
reference_labels = reference_labels.iloc[2*reference_labels.shape[0]//3: , :]
results = results.iloc[2*results.shape[0]//3: , :]

print(reference_labels.head(), reference_labels.shape)
print(results.head(), results.shape)


In [ ]:
plot_umap(results, "logratio_RNAbyDNA", reference_labels, output_filename="CAGI ref genome on ref_chkpt")

In [ ]:
results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/cagi_whole_hic/snp_ckpt/embeddings.csv", index_col=0, header=None)
print(results.shape)

# Take only the mut_seqs which is the last 1/3 rows of the reference labels file.
results = results.iloc[2*results.shape[0]//3: , :]

print(reference_labels.head(), reference_labels.shape)
print(results.head(), results.shape)

plot_umap(results, "logratio_RNAbyDNA", reference_labels, output_filename="CAGI ref genome on snp_chkpt")

In [ ]:
results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/cagi_whole_hic/hic_ckpt/embeddings.csv", index_col=0, header=None)
print(results.shape)

# Take only the mut_seqs which is the last 1/3 rows of the reference labels file.
results = results.iloc[2*results.shape[0]//3: , :]

print(reference_labels.head(), reference_labels.shape)
print(results.head(), results.shape)

plot_umap(results, "logratio_RNAbyDNA", reference_labels, output_filename="CAGI ref genome on hic_chkpt")

In [ ]:
plot_umap(results, "logratio_RNAbyDNA", reference_labels, output_filename="CAGI ref genome on ref_chkpt")

# eQTL 

In [ ]:
input_directory="/proj/bmfm/users/hongyang/data/omics/genome/finetune_datasets/eqtl/ref_genome/"
reference_labels = pd.read_csv(input_directory + "test.csv")
reference_labels.head()


results = pd.read_csv("/proj/bmfm/users/hongyang/benchmarking/modernbert_snp/eqtl_ref_genome/embeddings.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "label", reference_labels, output_filename="eQTL ref genome")

In [ ]:
# input_directory="/proj/bmfm/users/hongyang/data/omics/genome/finetune_datasets/eqtl/ref_genome/"
# reference_labels = pd.read_csv(input_directory + "test.csv")
# reference_labels.head()

results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/eqtl_ref_genome/snp_ckpt/embeddings.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "label", reference_labels, output_filename="eQTL ref genome snp_ckpt")

In [ ]:
results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/eqtl_ref_genome/hic_ckpt/embeddings.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "label", reference_labels, output_filename="eQTL ref genome hic_ckpt")

In [ ]:
input_directory="/proj/bmfm/users/hongyang/data/omics/genome/finetune_datasets/eqtl/snp_genome/"
reference_labels = pd.read_csv(input_directory + "test.csv")
reference_labels.head()


results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/eqtl_snp_genome/ref_ckpt/embeddings.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "label", reference_labels, output_filename="eQTL snp genome ref_ckpt")

In [ ]:
results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/eqtl_snp_genome/snp_ckpt/embeddings.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "label", reference_labels, output_filename="eQTL snp genome snp_ckpt")

In [ ]:
# Special sanity check for Hongyang's run...
results = pd.read_csv("/proj/bmfm/users/hongyang/biomed-multi-omic/tbl/embeddings_eqtl_pretrain_snpckpt_snpgenome.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "label", reference_labels, output_filename="eQTL snp genome snp_ckpt")

In [ ]:
results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/eqtl_snp_genome/hic_ckpt/embeddings.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "label", reference_labels, output_filename="eQTL snp genome hic_ckpt")

# Run K562 Mpra data

In [ ]:
input_directory="/proj/bmfm/datasets/omics/genome/finetune_datasets/lenti_mpra_regression/K562_original/"
reference_labels = pd.read_csv(input_directory + "test.csv")
reference_labels.head()

results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/mpra_k562_test/embeddings.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "mean_value", reference_labels)

In [ ]:
input_directory="/proj/bmfm/datasets/omics/genome/finetune_datasets/lenti_mpra_regression/K562_original/"
reference_labels = pd.read_csv(input_directory + "test.csv")
reference_labels.head()

results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/mpra_k562_test/snpckpt/embeddings.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "mean_value", reference_labels)

In [ ]:
# Compute the hic based embeddings now...
input_directory="/proj/bmfm/datasets/omics/genome/finetune_datasets/lenti_mpra_regression/K562_original/"
reference_labels = pd.read_csv(input_directory + "test.csv")
reference_labels.head()

results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/mpra_k562_test/hic_ckpt/embeddings.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "mean_value", reference_labels)

In [ ]:
# Compute the hic based embeddings now...
input_directory="/proj/bmfm/datasets/omics/genome/finetune_datasets/lenti_mpra_regression/K562_biallelic_200/"
reference_labels = pd.read_csv(input_directory + "test.csv")
reference_labels.head()

results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/mpra_k562_biallelic_test/hic_ckpt/embeddings.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "mean_value", reference_labels)

# Repeat this for promoter hic now...

In [ ]:
# Compute the hic based embeddings now...
input_directory="/proj/bmfm/datasets/omics/genome/finetune_datasets/promoter_prediction/DNA_BERT2/" 
reference_labels = pd.read_csv(input_directory + "test.csv")
reference_labels.head()

results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/promoter_dnabert2_original_test/hic_ckpt/embeddings.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "label", reference_labels)

In [ ]:
# Compute the hic based embeddings now...
input_directory="/proj/bmfm/datasets/omics/genome/finetune_datasets/promoter_prediction/DNA_BERT2/snpified_v1/snp_genome/" 
reference_labels = pd.read_csv(input_directory + "test.csv")
reference_labels.head()

results = pd.read_csv("/proj/bmfm/users/sanjoy/output_dir/promoter_dnabert2_snpified_v1_test/hic_ckpt/embeddings.csv", index_col=0, header=None)
print(results.head(), results.shape)

plot_umap(results, "label", reference_labels)